In [87]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn import linear_model

In [233]:
data = pd.read_csv('sample_forecasting_data.csv')
forecast_period = pd.read_csv('foreacast_period.csv')
len_FP = len(forecast_period)

In [234]:
data.head(3)

,Month,year,JobCode,Count,Year_Month
0,1,2016,A,14,2016_01
1,2,2016,A,16,2016_02
2,3,2016,A,19,2016_03


In [235]:
forecast_period.head()

,Month,year,JobCode,Year_Month,Count
0,11,2016,NaN,2017_11,NaN
1,12,2016,NaN,2017_12,NaN


In [224]:
def empty_df():
    empty = pd.DataFrame({'Month' : [np.nan], 'year' : [np.nan] , 'JobCode' : [np.nan], 'Year_Month' : [np.nan], 'Count' : [np.nan], 'MA' : [np.nan]})
    empty = empty[['Month', 'year', 'JobCode', 'Year_Month', 'Count']]
    empty = empty.iloc[1:]
    
    return empty

In [180]:
def data_split(finaldata):
    cut = len(finaldata)-1
    training_set = finaldata[0:cut]
    test_set = finaldata[cut:]
    
    # setting up training and test set as numpy arrays
    training_set_features = training_set[['Month', 'MA']].values
    training_set_labels = training_set['Count'].values
    test_set_features = test_set[['Month', 'MA']].values
    #test_set_labels = test_set['Count'].values
    
    test_set_features
    
    return training_set_features, training_set_labels, test_set_features

In [125]:
def linmod(x, y ,z):
    reg = linear_model.LinearRegression(normalize=True)

    # fitting the training set in the model
    reg.fit(x,y)
    pred = reg.predict(z)
    
    return pred

In [227]:
combined = empty_df()
for category in data['JobCode'].unique():
    df_slice = data[data['JobCode'] == category]
    forecast_period['JobCode'] = category 
    for i in range(len_FP):
        next_period = forecast_period.iloc[[i]]
        df_slice = pd.concat([df_slice, next_period])
        df_slice.reset_index(inplace = True)
        df_slice['MA'] = df_slice['Count'].rolling(min_periods=1,window=3).mean().shift(1)
            
        x1, y1, x2 = data_split(df_slice[1:])
        prediction = linmod(x1,y1,x2)
            
        df_slice = df_slice[['Month', 'year', 'JobCode', 'Year_Month', 'Count', 'MA']]
        df_slice.loc[len(df_slice) - 1,'Count'] = round(prediction,0)
    combined = pd.concat([combined, df_slice])
combined = combined[['Month', 'year', 'JobCode', 'Year_Month', 'Count', 'MA']]
    

In [236]:
combined

,Month,year,JobCode,Year_Month,Count,MA
0,1.0,2016.0,A,2016_01,14.0,NaN
1,2.0,2016.0,A,2016_02,16.0,14.000000
2,3.0,2016.0,A,2016_03,19.0,15.000000
3,4.0,2016.0,A,2016_04,11.0,16.333333
4,5.0,2016.0,A,2016_05,20.0,15.333333
5,6.0,2016.0,A,2016_06,11.0,16.666667
6,7.0,2016.0,A,2016_07,19.0,14.000000
7,8.0,2016.0,A,2016_08,20.0,16.666667
8,9.0,2016.0,A,2016_09,12.0,16.666667
9,10.0,2016.0,A,2016_10,19.0,17.000000
